# Episodes

Prints episode information from the Real Housewives franchise.

## Import Libraries

In [3]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
url_list = pd.read_csv('./data/season_urls.csv')

## Scrape Table Data and Print

In [ ]:
# Loop through the list of URLs for each series.
for index, row in url_list.iterrows():
    series = row['Series']
    soup = BeautifulSoup(urlopen(row['URL']))
    tables = soup.find_all(class_='wikiepisodetable')

    # Loop through each table summarizing one season of the series.
    for table in tables:
        season_number = table.caption.text
        season_number = season_number.replace(' episodes', '')
        season_number = season_number[-2:].strip()
        rows = table.find_all('tr', class_='vevent')

        # Loop through each row summarizing one episode of the season.
        for row in rows:
            tds = row.find_all('td')
            episode_number = tds[0].text

            # Format string to remove quotation marks that border the title
            # then replace apostrophes with double apostrophes to avoid SQL syntax errors.
            title = tds[1].text[1:-1]
            title = title.replace("'", "''")

            # Format string to remove parentheses that border the date.
            date = tds[2].span.text.strip()
            date = date[1:-1]

            print(
f"""(
    (
        SELECT
            season_id
        FROM
            seasons
        WHERE
            series_id = '{series}'
            AND season_number = {season_number}
    ),
    {episode_number},
    '{title}',
    '{date}'
),""")